In [1]:
### Imports for the Data Preprocessing
import numpy as np
import pandas as pd
import matplotlib as plt
import os

In [2]:
# get working directory and remove last folder
# TODO: make this more robust
wd = os.path.dirname(os.getcwd())
os.chdir(wd)
print('Working Directory: ', os.getcwd())

Working Directory:  c:\Users\Stephan\Documents\Python\Python-Stat\Neuer\Kaggle_semiinar\student-performance


In [3]:
dtypes={
    'elapsed_time':np.int32,
    'event_name':'category',
    'name':'category',
    'level':np.uint8,
    'room_coor_x':np.float32,
    'index':np.int32,
    'room_coor_y':np.float32,
    'screen_coor_x':np.float32,
    'screen_coor_y':np.float32,
    'hover_duration':np.float32,
    'text':'category',
    'fqid':'category',
    'room_fqid':'category',
    'text_fqid':'category',
    'fullscreen':'category',
    'hq':'category',
    'music':'category',
    'level_group':'category'}


# os.chdir('N:\MASTER_DS\Code\Kaggle_competition\Kaggle-seminar\student-performance')
dataset_df = pd.read_csv('data/raw/train.csv', dtype=dtypes, nrows= 20000)

In [4]:
#preprocessing function to add variables and rescale the time 
def adding_new_variables_rescaling(dataset_df):
    dataset_df = dataset_df.sort_values(['session_id','elapsed_time'])
    dataset_df['elapsed_time'] = dataset_df['elapsed_time']/1000
    group = dataset_df.groupby(['session_id','level'])['elapsed_time'].diff()
    group = group.fillna(value= 0)
    dataset_df= dataset_df.assign(difference_clicks = group)

    return dataset_df

In [5]:
#preprocessing to combine variables on level stage
#Function to clean the sequential data for the training of the model

#For that Function to work we need to specify the variables in Categorical and Numerical & Counting

CATEGORICAL = ['event_name', 'name', 'fqid', 'room_fqid', 'text_fqid', 'fullscreen', 'hq', 'music']
NUMERICALmean = ['hover_duration','difference_clicks']
NUMERICALstd = ['elapsed_time','page','room_coor_x', 'room_coor_y', 'screen_coor_x', 'screen_coor_y', 'hover_duration', 'difference_clicks']
COUNTING = ['index']
MAXIMUM = ['difference_clicks', 'elapsed_time']



def feature_engineer_steve(dataset_df):
    dfs = []
    tmp = dataset_df.groupby(['session_id','level'])["level_group"].first()
    tmp.name = tmp.name 
    dfs.append(tmp)
    for c in CATEGORICAL:
        if c not in ['fullscreen', 'hq', 'music']:
            tmp = dataset_df.groupby(['session_id','level'])[c].agg('nunique')
        else:
            tmp = dataset_df.groupby(['session_id','level'])[c].first().astype(int).fillna(0)
        dfs.append(tmp)
    for c in NUMERICALmean:
        tmp = dataset_df.groupby(['session_id','level'])[c].agg('mean')
        tmp.name = tmp.name + '_mean'
        dfs.append(tmp)
    for c in NUMERICALstd:
        tmp = dataset_df.groupby(['session_id','level'])[c].agg('std')
        tmp.name = tmp.name + '_std'
        dfs.append(tmp)    
    for c in COUNTING:
        tmp = 1+ dataset_df.groupby(['session_id','level'])[c].agg('max')- dataset_df.groupby(['session_id','level'])[c].agg('min') 
        tmp.name = tmp.name + '_sum_of_actions'
        dfs.append(tmp)
    for c in MAXIMUM:
        tmp = dataset_df.groupby(['session_id','level'])[c].agg('max')- dataset_df.groupby(['session_id','level'])[c].agg('min') 
        tmp.name = tmp.name + '_max'
        dfs.append(tmp)
    
    dataset_df = pd.concat(dfs,axis=1)
    dataset_df = dataset_df.fillna(-1)
    dataset_df = dataset_df.reset_index()
    dataset_df = dataset_df.set_index('session_id') 
    
# add Clicks per second afterwards cause we need the time for each level first
    dataset_df['clicks_per_second'] = dataset_df['index_sum_of_actions']/ dataset_df['elapsed_time_max']
 
    return dataset_df

In [6]:
#return the preprocessed data
##test data preprocessing with Subset of 5 million rows

dataset_df_added = adding_new_variables_rescaling(dataset_df)
dataset_df_level = feature_engineer_steve(dataset_df_added)
print(dataset_df_added)
print(dataset_df_level)

              session_id  index  elapsed_time          event_name       name  \
0      20090312431273200      0         0.000      cutscene_click      basic   
2      20090312431273200      2         0.831        person_click      basic   
3      20090312431273200      3         1.147        person_click      basic   
1      20090312431273200      1         1.323        person_click      basic   
4      20090312431273200      4         1.863        person_click      basic   
...                  ...    ...           ...                 ...        ...   
19995  20090317080721164   1364      2454.048  notification_click      basic   
19996  20090317080721164   1365      2455.479        object_click      basic   
19997  20090317080721164   1366      2456.860        object_hover  undefined   
19998  20090317080721164   1367      2457.726        object_click      basic   
19999  20090317080721164   1368      2459.226  notification_click      basic   

       level  page  room_coor_x  room_c

In [ ]:
# Export results
#dataset_df_added.to_csv('data/processed/data_df_added.csv')
dataset_df_level.to_csv('data/processed/data_df_level.csv')

In [ ]:
print(dataset_df_level.dtypes)

In [8]:
dataset_df_level.columns

Index(['level', 'level_group', 'event_name', 'name', 'fqid', 'room_fqid',
       'text_fqid', 'fullscreen', 'hq', 'music', 'hover_duration_mean',
       'difference_clicks_mean', 'elapsed_time_std', 'page_std',
       'room_coor_x_std', 'room_coor_y_std', 'screen_coor_x_std',
       'screen_coor_y_std', 'hover_duration_std', 'difference_clicks_std',
       'index_sum_of_actions', 'difference_clicks_max', 'elapsed_time_max',
       'clicks_per_second'],
      dtype='object')

In [16]:
# Group the dataframe by level_group column
dataset_df.groupby(['session_id','level'])
dataset_df_level.groupby("session_id",'dataset_df_level.groupby("session_id",'level_group')
')
print(grouped)

'''# Create an empty dataframe to store the concatenated results
result = pd.DataFrame()

# Loop over each group
for name, group in grouped:
    # Remove rows with missing values
    group = group.dropna()
    
    # Create a new row by concatenating all rows in the group
    new_row = pd.concat(list(group.iloc[:, 2:]), axis=0)
    
    # Append the new row to the result dataframe
    result = result.append(new_row, ignore_index=True)

    # Add the level_group value to the result dataframe
    result.loc[result.index[-1], 'level_group'] = name

'''

ValueError: No axis named level_group for object type DataFrame